In [10]:
PATH_TO_DATA = ''

In [114]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

Подготовим словарь с названиями фильмов, чтобы быстро находить название по movieId

In [3]:
dict_of_titles = dict()
with open(PATH_TO_DATA + 'movies.csv') as f:
    next(f)
    for line in f:
        line_splited = line.split(',')
        movieId = int(line_splited[0])
        title = line_splited[1]
        dict_of_titles[movieId] = title

# Подготовка данных.

### 1. Создадим датафрейм с нашими рейтингами.

In [4]:
ratings = pd.read_csv(PATH_TO_DATA + 'ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### 2. Разделим нашу выборку на train (80%) и test (20%)

In [230]:
ratings = ratings.sample(frac=1)

In [232]:
train_size = int(ratings.shape[0] * 0.8)

In [233]:
ratings_train = ratings.iloc[0: train_size].copy(deep=True)
ratings_test = ratings.iloc[train_size:].copy(deep=True)

In [234]:
ratings_train.shape

(80668, 4)

In [235]:
ratings_test.shape

(20168, 4)

### 3. Подготовим метрику для оценки качества наших рекомендаций

In [236]:
# сохраним тестовые рейтинги в словарь {userId: set(movieId)}
ratings_test_dict = ratings_test.groupby('userId')['movieId'].agg(set).to_dict()

In [237]:
def avg_precision_topK(res, pred, k=5):
    total_precision = list()
    for uid in pred:
        if uid in res:
            pred_uid = sorted(pred[uid], key=lambda x: -x[1])[0: k]
            user_precision = len(set([movie for (movie, score) in pred_uid]) & res[uid]) / k
            total_precision.append(user_precision)
    return np.mean(total_precision)

In [238]:
pred = {15: [(47, 0.8), (260, 0.5), (11, 0.3)]}

In [239]:
avg_precision_topK(ratings_test_dict, pred, 3)

0.0

# Неперсонализированые рекомендации

### 1. Найдем топ фильмов по кол-ву оценок.

In [14]:
ratings_train.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [308]:
items_popularity = ratings_train.groupby('movieId')['userId'].count().to_dict()

In [19]:
# в rec сохраняем list из пар (movieId, его популярность). Берем только топ 20 самых популярных фильмов
rec = ???

In [1]:
# смотрим что это за фильмы
print([dict_of_titles[movieId] for movieId, _ in rec])

In [311]:
# формируем словарь с рекомендациями для каждого пользователя
rec_dict = {uid: rec for uid in ratings_test_dict}

In [2]:
# считаем нашу метрику
avg_precision_topK(ratings_test_dict, rec_dict, 20)

### 2. Найдем топ фильмов по среднему рейтингу.

In [313]:
items_mean_rating = ratings_train.groupby('movieId')['rating'].mean().to_dict()

In [314]:
# в rec сохраняем list из пар (movieId, его средний рейтинг). Берем только топ 20 фильмов с самым высоким средним рейтингом
rec = ???

In [3]:
# # смотрим что это за фильмы
print([dict_of_titles[movieId] for movieId, _ in rec])

In [316]:
rec_dict = {uid: rec for uid in ratings_test_dict}

In [4]:
avg_precision_topK(ratings_test_dict, rec_dict, 20)

### 3. Найдем топ фильмов по рейтингу с регуляризацией.

In [318]:
# сохраняем средний рейтинг для всех фильмов
mu = ???

In [319]:
ratings_train_grouped = ratings_train.groupby('movieId').agg({'rating': 'sum', 'userId': 'count'})
ratings_train_grouped.columns = ['movie_rating_sum', 'movie_popularity']

In [320]:
k = 5

In [322]:
ratings_train_grouped['mean_rating_regularized'] = ???

In [323]:
items_mean_rating = ratings_train_grouped['mean_rating_regularized'].to_dict()

In [324]:
rec = sorted(items_mean_rating.items(), key=lambda x: -x[1])[0: 20]

In [5]:
print([dict_of_titles[movieId] for movieId, _ in rec])

In [326]:
rec_dict = {uid: rec for uid in ratings_test_dict}

In [6]:
avg_precision_topK(ratings_test_dict, rec_dict, 20)

### 3.1 Подобрать оптимальное значение параметра k. Построить график зависимости метрики качества от значения параметра k.

Положим весь код из предыдущего шага в функцию compute_metric.  
Она должна принимать на вход параметр k - и возвращать значение нашей метрики.

In [260]:
def compute_metric(k):
    ???

In [261]:
k_values = list(range(1, 40))

In [262]:
precision_values = [compute_metric(k) for k in k_values]

In [7]:
plt.plot(k_values, precision_values)
plt.xlabel('k value')
plt.ylabel('precision at top 20')
plt.show()

In [266]:
compute_metric(30)

0.0674055829228243

# User-based и Item-based подходы

In [267]:
from sklearn.metrics.pairwise import cosine_similarity

In [284]:
max_userId = ratings['userId'].max()

In [286]:
max_movieId = ratings['movieId'].max()

In [287]:
user_item_matrix = np.zeros([max_userId+1, max_movieId+1])

In [288]:
for row in ratings_train.itertuples():
    userId = row[1]
    movieId = row[2]
    rating = row[3]
    user_item_matrix[userId, movieId] = rating

### 1. Написать функцию для поиска топ-K ближайших юзеров

In [289]:
def find_topK_similar_users(uid, k):
    cosine_similarity_between_users = cosine_similarity(user_item_matrix[uid].reshape(1, -1), user_item_matrix)
    most_similar_indices = np.argsort(cosine_similarity_between_users[0])[-2: -k: -1]
    ans = list(zip(most_similar_indices, cosine_similarity_between_users[0, most_similar_indices]))
    return ans

In [290]:
find_topK_similar_users(1, 10)

[(590, 0.3007785692667796),
 (266, 0.2918266905829427),
 (469, 0.29042638532345144),
 (57, 0.28650404766661997),
 (45, 0.2862911029141325),
 (39, 0.2842449328333725),
 (313, 0.28391379016374685),
 (91, 0.2824610201866022)]

### 2. Написать функцию для поиска топ-K ближайших айтемов

In [291]:
def find_topK_similar_items(item_id, k):
    ???

In [8]:
dict_of_titles[260]

In [293]:
sim_items = find_topK_similar_items(260, 10)

In [9]:
print([(movieId, dict_of_titles[movieId]) for movieId, _ in sim_items])

### 3. Написать рекомендатель для пользователя на основе Item-based подхода

In [295]:
def return_user_recommendations(uid, k):
    user_ratings = ratings_train[ratings_train['userId'] == uid].copy(deep=True)
    user_ratings['rating'] = user_ratings['rating'] - user_ratings['rating'].mean() + 0.001
    user_recommendations = np.zeros(max_movieId+1)
    user_recommendations_divisor = np.zeros(max_movieId+1)
    user_recommendations_divisor += 0.001
    for row in user_ratings.iterrows():
        movieId = int(row[1][1])
        rating = row[1][2]
        for iid, score in find_topK_similar_items(movieId, 10):
            user_recommendations[iid] += score*rating
            user_recommendations_divisor[iid] += score
    user_recommendations = user_recommendations / user_recommendations_divisor
    most_similar_indices = np.argsort(user_recommendations)[-1: -k: -1]
    ans = list(zip(most_similar_indices, user_recommendations[most_similar_indices]))
    return ans

In [296]:
%%time
rec = return_user_recommendations(15, 20)

CPU times: user 1min 26s, sys: 38.2 s, total: 2min 4s
Wall time: 1min 50s


In [297]:
def avg_precision_topK(res, pred, k=5):
    total_precision = list()
    for uid in pred:
        if uid in res:
            pred_uid = sorted(pred[uid], key=lambda x: -x[1])[0: k]
            user_precision = len(set([movie for (movie, score) in pred_uid]) & res[uid]) / k
            total_precision.append(user_precision)
    return np.mean(total_precision)

In [298]:
avg_precision_topK(ratings_test_dict, {15: rec}, 20)

0.0

# ALS

In [299]:
from implicit.als import AlternatingLeastSquares

In [300]:
from scipy.sparse import csr_matrix

In [301]:
user_item_matrix_csr = csr_matrix(user_item_matrix)

In [302]:
model = AlternatingLeastSquares(factors=50)

# обучаем модель (нужна матрица item-user)
model.fit(user_item_matrix_csr.T)

100%|██████████| 15.0/15 [00:00<00:00, 18.74it/s]


In [303]:
# recommend items for a user
recommendations = model.recommend(15, user_item_matrix_csr, N=20)

In [305]:
res = dict()
for uid in ratings_test_dict:
    try:
        res[uid] = model.recommend(uid, user_item_matrix_csr, N=20)
    except IndexError:
        continue

In [1]:
avg_precision_topK(ratings_test_dict, res, 20)

### Подобрать оптимальные параметры для ALS (factors и regularization)